In [5]:
import pymongo
import json

In [6]:
client = pymongo.MongoClient('mongodb://127.0.0.1:27017/')
db = client["student"]
information = db.chart
file = open("C:/Users/Snega/Downloads/students.json")
data = json.load(file)

In [318]:
data

[{'_id': 0,
  'name': 'aimee Zank',
  'scores': [{'score': 1.463179736705023, 'type': 'exam'},
   {'score': 11.78273309957772, 'type': 'quiz'},
   {'score': 35.8740349954354, 'type': 'homework'}]},
 {'_id': 1,
  'name': 'Aurelia Menendez',
  'scores': [{'score': 60.06045071030959, 'type': 'exam'},
   {'score': 52.79790691903873, 'type': 'quiz'},
   {'score': 71.76133439165544, 'type': 'homework'}]},
 {'_id': 2,
  'name': 'Corliss Zuk',
  'scores': [{'score': 67.03077096065002, 'type': 'exam'},
   {'score': 6.301851677835235, 'type': 'quiz'},
   {'score': 66.28344683278382, 'type': 'homework'}]},
 {'_id': 3,
  'name': 'Bao Ziglar',
  'scores': [{'score': 71.64343899778332, 'type': 'exam'},
   {'score': 24.80221293650313, 'type': 'quiz'},
   {'score': 42.26147058804812, 'type': 'homework'}]},
 {'_id': 4,
  'name': 'Zachary Langlais',
  'scores': [{'score': 78.68385091304332, 'type': 'exam'},
   {'score': 90.2963101368042, 'type': 'quiz'},
   {'score': 34.41620148042529, 'type': 'homework

In [191]:
if isinstance(data, list):
    information.insert_many(data)
else:
    information.insert_one(data)

In [319]:
# student scores maximum out of all type of exam categories
for i in information.aggregate([{"$project": {"_id": "$name", "max_mark": {"$max": "$scores.score"}}}]):
    print(i)

{'_id': 'aimee Zank', 'max_mark': 35.8740349954354}
{'_id': 'Aurelia Menendez', 'max_mark': 71.76133439165544}
{'_id': 'Corliss Zuk', 'max_mark': 67.03077096065002}
{'_id': 'Bao Ziglar', 'max_mark': 71.64343899778332}
{'_id': 'Zachary Langlais', 'max_mark': 90.2963101368042}
{'_id': 'Wilburn Spiess', 'max_mark': 63.42288310628662}
{'_id': 'Jenette Flanders', 'max_mark': 81.57115318686338}
{'_id': 'Salena Olmos', 'max_mark': 96.52986171633331}
{'_id': 'Daphne Zheng', 'max_mark': 75.94123677556644}
{'_id': 'Sanda Ryba', 'max_mark': 97.80449632538915}
{'_id': 'Denisha Cast', 'max_mark': 98.35723209418343}
{'_id': 'Marcus Blohm', 'max_mark': 87.49924733328717}
{'_id': 'Quincy Danaher', 'max_mark': 85.61270164694737}
{'_id': 'Jessika Dagenais', 'max_mark': 95.17753772405909}
{'_id': 'Alix Sherrill', 'max_mark': 68.64484047692098}
{'_id': 'Tambra Mercure', 'max_mark': 69.1565022533158}
{'_id': 'Dodie Staller', 'max_mark': 80.52136407989194}
{'_id': 'Fletcher Mcconnell', 'max_mark': 97.701166

In [193]:
# student name who scored the highest of all
for i in information.aggregate([{
    "$project":  {"_id": "$name", "max_mark": {"$max": "$scores.score"}
    }
},
{
    "$sort": {
        "max_mark": pymongo.DESCENDING,
    }
},
    {"$limit":1,
    
    },

]):
    print(i)

{'_id': 'Cody Strouth', 'max_mark': 99.80348240553108}


In [194]:
# student scored below 40 percent in the exam
for i in information.aggregate([ 
   {
      "$project": {"_id": "$name",
          "scores": {
             "$filter": {
                "input": "$scores",
               "as": "mark",
               "cond": { "$and":[
                   {"$eq": [ "$$mark.type", "exam" ] },
                   {"$lte": [ "$$mark.score", 40 ] }]}
             },
            } 
      }
   }
]):
    print(i)



{'_id': 'aimee Zank', 'scores': [{'score': 1.463179736705023, 'type': 'exam'}]}
{'_id': 'Aurelia Menendez', 'scores': []}
{'_id': 'Corliss Zuk', 'scores': []}
{'_id': 'Bao Ziglar', 'scores': []}
{'_id': 'Zachary Langlais', 'scores': []}
{'_id': 'Wilburn Spiess', 'scores': []}
{'_id': 'Jenette Flanders', 'scores': [{'score': 37.32285459166097, 'type': 'exam'}]}
{'_id': 'Salena Olmos', 'scores': []}
{'_id': 'Daphne Zheng', 'scores': [{'score': 22.13583712862635, 'type': 'exam'}]}
{'_id': 'Sanda Ryba', 'scores': []}
{'_id': 'Denisha Cast', 'scores': []}
{'_id': 'Marcus Blohm', 'scores': []}
{'_id': 'Quincy Danaher', 'scores': []}
{'_id': 'Jessika Dagenais', 'scores': []}
{'_id': 'Alix Sherrill', 'scores': [{'score': 25.15924151998215, 'type': 'exam'}]}
{'_id': 'Tambra Mercure', 'scores': []}
{'_id': 'Dodie Staller', 'scores': [{'score': 7.772386442858281, 'type': 'exam'}]}
{'_id': 'Fletcher Mcconnell', 'scores': [{'score': 39.41011069729274, 'type': 'exam'}]}
{'_id': 'Verdell Sowinski', '

In [195]:
# labeling the result(pass or fail) for each student based on the condition
for i in information.aggregate([
    {
        "$addFields": {
            "scores": {
                "$map": {
                    "input": "$scores",
                    "in": {
                        "$mergeObjects": [
                            "$$this",
                            { 
                                "Result": {
                                    "$cond": {"if": { "$lte": ['$$this.score', 40]},"then": "Fail","else": "Pass"
                                    }
                                }  
                            }
                        ]
                    }
                }
            }
        }
    }
]):
    print(i)
    


{'_id': 0, 'name': 'aimee Zank', 'scores': [{'score': 1.463179736705023, 'type': 'exam', 'Result': 'Fail'}, {'score': 11.78273309957772, 'type': 'quiz', 'Result': 'Fail'}, {'score': 35.8740349954354, 'type': 'homework', 'Result': 'Fail'}]}
{'_id': 1, 'name': 'Aurelia Menendez', 'scores': [{'score': 60.06045071030959, 'type': 'exam', 'Result': 'Pass'}, {'score': 52.79790691903873, 'type': 'quiz', 'Result': 'Pass'}, {'score': 71.76133439165544, 'type': 'homework', 'Result': 'Pass'}]}
{'_id': 2, 'name': 'Corliss Zuk', 'scores': [{'score': 67.03077096065002, 'type': 'exam', 'Result': 'Pass'}, {'score': 6.301851677835235, 'type': 'quiz', 'Result': 'Fail'}, {'score': 66.28344683278382, 'type': 'homework', 'Result': 'Pass'}]}
{'_id': 3, 'name': 'Bao Ziglar', 'scores': [{'score': 71.64343899778332, 'type': 'exam', 'Result': 'Pass'}, {'score': 24.80221293650313, 'type': 'quiz', 'Result': 'Fail'}, {'score': 42.26147058804812, 'type': 'homework', 'Result': 'Pass'}]}
{'_id': 4, 'name': 'Zachary La

In [10]:
# Total and average score of each student
list =[]
for i in information.aggregate([
    {"$project": {"_id": "$name", 
                  "totalmark": {"$sum": "$scores.score"}, 
                  "average":{"$avg":"$scores.score"}}}]):
    print(i)

{'_id': 'aimee Zank', 'totalmark': 49.11994783171814, 'average': 16.373315943906046}
{'_id': 'Aurelia Menendez', 'totalmark': 184.61969202100374, 'average': 61.53989734033458}
{'_id': 'Corliss Zuk', 'totalmark': 139.61606947126907, 'average': 46.53868982375636}
{'_id': 'Bao Ziglar', 'totalmark': 138.70712252233457, 'average': 46.23570750744486}
{'_id': 'Zachary Langlais', 'totalmark': 203.3963625302728, 'average': 67.79878751009093}
{'_id': 'Wilburn Spiess', 'totalmark': 134.0186975547794, 'average': 44.67289918492647}
{'_id': 'Jenette Flanders', 'totalmark': 147.2203575476617, 'average': 49.07345251588723}
{'_id': 'Salena Olmos', 'totalmark': 229.3959334774732, 'average': 76.46531115915774}
{'_id': 'Daphne Zheng', 'totalmark': 112.71677331754347, 'average': 37.57225777251449}
{'_id': 'Sanda Ryba', 'totalmark': 220.08328118626565, 'average': 73.36109372875522}
{'_id': 'Denisha Cast', 'totalmark': 199.8843572885121, 'average': 66.62811909617069}
{'_id': 'Marcus Blohm', 'totalmark': 248.

In [280]:

for i in information.aggregate( [
    {"$group": {"_id": "$name", "max_mark": {"$sum": "$scores.score"}}},
    {"$project":{"totalmark":{"$sum":"$max_mark"},"Average":{"$avg":"$max_mark"}}},
    {"$sort":{"_id":1}},
    {"$out": "Totalmark"}    
   ] ):
    print(i)

In [198]:
# students who scored greater than 40
for i in information.aggregate([ 
   {
      "$project": {"_id": "$name",
          "scores": {
             "$filter": {
                "input": "$scores",
               "as": "mark",
               "cond":{"$gte": [ "$$mark.score", 40 ] }
             },
            } 
      }
   },
 
]):
    print(i)

{'_id': 'aimee Zank', 'scores': []}
{'_id': 'Aurelia Menendez', 'scores': [{'score': 60.06045071030959, 'type': 'exam'}, {'score': 52.79790691903873, 'type': 'quiz'}, {'score': 71.76133439165544, 'type': 'homework'}]}
{'_id': 'Corliss Zuk', 'scores': [{'score': 67.03077096065002, 'type': 'exam'}, {'score': 66.28344683278382, 'type': 'homework'}]}
{'_id': 'Bao Ziglar', 'scores': [{'score': 71.64343899778332, 'type': 'exam'}, {'score': 42.26147058804812, 'type': 'homework'}]}
{'_id': 'Zachary Langlais', 'scores': [{'score': 78.68385091304332, 'type': 'exam'}, {'score': 90.2963101368042, 'type': 'quiz'}]}
{'_id': 'Wilburn Spiess', 'scores': [{'score': 44.87186330181261, 'type': 'exam'}, {'score': 63.42288310628662, 'type': 'homework'}]}
{'_id': 'Jenette Flanders', 'scores': [{'score': 81.57115318686338, 'type': 'homework'}]}
{'_id': 'Salena Olmos', 'scores': [{'score': 90.37826509157176, 'type': 'exam'}, {'score': 42.48780666956811, 'type': 'quiz'}, {'score': 96.52986171633331, 'type': 'h

In [320]:
# students who scored less than 40
for i in information.aggregate([ 
   {
      "$project": {"_id": "$name",
          "scores": {
             "$filter": {
                "input": "$scores",
               "as": "mark",
               "cond":{"$lte": [ "$$mark.score", 40 ] }
             },
             
            } 
      }
   },
]):
    print(i)

{'_id': 'aimee Zank', 'scores': [{'score': 1.463179736705023, 'type': 'exam'}, {'score': 11.78273309957772, 'type': 'quiz'}, {'score': 35.8740349954354, 'type': 'homework'}]}
{'_id': 'Aurelia Menendez', 'scores': []}
{'_id': 'Corliss Zuk', 'scores': [{'score': 6.301851677835235, 'type': 'quiz'}]}
{'_id': 'Bao Ziglar', 'scores': [{'score': 24.80221293650313, 'type': 'quiz'}]}
{'_id': 'Zachary Langlais', 'scores': [{'score': 34.41620148042529, 'type': 'homework'}]}
{'_id': 'Wilburn Spiess', 'scores': [{'score': 25.72395114668016, 'type': 'quiz'}]}
{'_id': 'Jenette Flanders', 'scores': [{'score': 37.32285459166097, 'type': 'exam'}, {'score': 28.32634976913737, 'type': 'quiz'}]}
{'_id': 'Salena Olmos', 'scores': []}
{'_id': 'Daphne Zheng', 'scores': [{'score': 22.13583712862635, 'type': 'exam'}, {'score': 14.63969941335069, 'type': 'quiz'}]}
{'_id': 'Sanda Ryba', 'scores': [{'score': 25.27368532432955, 'type': 'homework'}]}
{'_id': 'Denisha Cast', 'scores': []}
{'_id': 'Marcus Blohm', 'sco